<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_Time_weighted_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

RAG는 여러 과정을 거치면서 진행된다. 매 과정이 치명적으로 중요하다.

- 멀티쿼리 : 대충질문해도 좋은 답변을 원하는 이들을 위해 다음에 집중해야 한다.
- 페어런트 도큐먼트 : 앞뒤 문장을 잘 담아야 하고(chunk의 중요성)
- 셀프쿼리(질문재해석) : 시맨틱검색 말고 쿼리가 필요한 경우
  - 시맨틱검색이란 질문문장과 임베딩데이터의 벡터값에 따라 유사데이터를 걸러내는 것인데, 이 경우 질문의 모양이 조금만 달라져도 추출 데이터 자체가 달라진다. 이것은 '질문이 우선되는 구조'이기 때문이다.
  이 경우, 데이터를 중심으로 사용자의 질문을 참고하여 쿼리를 날려서 정리해야 할 필요가 생길 수 있다. 이것도 고려해야 한다.
- 타임 웨이티드 : 오래된 자료는 덜 참고했으면...
  - 최근에 올라간 자료에 무게를 둬서 답변을 참고했으면 좋겠다.

## Time-weighted vector Retriever

제목 그 자체가 스포인데, 추출시 데이터의 신선도를 반영하는 리트리버이다.
'시간이 지난만큼' 패널티를 주자...는 것.

[ 시맨틱유사성 + (1.0 - 경과시간에 따른 부패도) ]
따라서 만약 부패도반영을 1.0을 해버리면... 걍 일반 리트리버랑 똑같아지는거지.

- [ ] 그럼... 그냥 self~ 는 시간을 전혀 고려하지 않는다는건가? '최신에'라고 쿼리에 넣어도???
- [ ] 그럼 메타를 붙일 때 시간값을 추가해주면 될텐데... 등록시간 기준인거야 아니면 데이터 그 자체의 신선도인거야? 예를 들어... 2024년에 만들어진 1999년의 경제보고서의 데이터가 우선되는 것인지, 2023년에 만들어진 2022년의 보고서데이터가 우선되는 것인지? 아마 옵션으로 주는게 있을텐데...
- [ ] 데이터를 넣어 줄 때 이 모든 걸 다 찾아서 넣어줘야 한다고? 장난해...?


### 결론.
**decay rate와 relevance 사이의 절묘한 줄타기가 필요한 부분이당**

## Setting

In [1]:
# colab 환경설정
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install -U -q langchain pypdf sentence_transformers chromadb langchain-openai
!pip install -q langchain langchain-openai sentence_transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
# 유틸들
import math
import time

def trace(func):
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print(f"{end - start:.5f} sec")
    return wrapper

In [9]:
# ___ setting ___
LLM_MODEL = "gpt-3.5-turbo"
MAX = 265
TEMP = 1.5
# _______________

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings

openai_compbot = OpenAI(
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)
openai_chatbot = ChatOpenAI(
    model_name = LLM_MODEL,
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)

MODEL_EMBED= "jhgan/ko-sbert-nli"

ko_embed = HuggingFaceEmbeddings(
    model_name= MODEL_EMBED,
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
    )

llm = openai_compbot
llm_chat = openai_chatbot

In [5]:
URL = 'https://n.news.naver.com/article/002/0002319323?cds=news_media_pc&type=editn'
URLs = [
    URL,
    'https://n.news.naver.com/article/079/0003862456?cds=news_media_pc&type=editn'
]
SOURCE_folder = "/content/drive/MyDrive/data/test"
SOURCE = "/content/source/1-1그로스 해킹, 마케팅과 어떻게 다른가요_ - PUBLY.pdf"

DB_URL = '/content/drive/MyDrive/vector_upgrade/'

# 코드

In [6]:
!pip install -U -q faiss-gpu # GPU사용시
# !pip install -U -q faiss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.9 MB/s eta 0:00:00


## Time-weighted vector Retriever

In [7]:
from datetime import datetime, timedelta

import faiss

from langchain_community.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore # 아... 이놈 나왔을 때 좋은 기억이 없는데 오늘도 평탄하진 않겠네...ㅡ.ㅡ;
from langchain.schema import Document # 이건 데이터에 대한 잔소리를 추가해줄 때 사용하는 모양.
from langchain.retrievers import TimeWeightedVectorStoreRetriever

In [19]:
# 임베딩 사이즈는 faiss에서 요구하는 값이며 모델마다 다를 수 있다.
# 현재 사용중인 'MODEL_EMBED= "jhgan/ko-sbert-nli"' 모델의 경우, https://huggingface.co/jhgan/ko-sbert-nli 에서
# Full Model Architecture > SentenceTransformer > Pooling > 'word_embedding_dimension': 768 임을 확인할 수 있다.
embedding_size = 768
index = faiss.IndexFlatL2(embedding_size) # L2는 뭐지? L2캐시 이야기인가? 아는게 그거밖에 없어서.
vectorstore = FAISS(
    ko_embed,
    index,
    InMemoryDocstore({}), # 역시나 임시저장소. 하지만 나중에 persist path랑 바꿔끼우기는 좋겠군.
    {} # 이 특이한 빈자리는 머지?
    )

In [25]:
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore,
    decay_rate = 0.99, # 시간가중치 최대
    # decay_rate = 0.01,
    k=1
)

In [26]:
yesterday = datetime.now() - timedelta(days=1)
retriever.add_documents([
    Document(
        page_content = '영어는 훌륭합니다',
        metadata={'last_accessed_at' : yesterday}
        )
    ]
)

retriever.add_documents([
    Document(
        page_content='한국어는 훌륭합니다'
    )
])

['29772347-4eea-40b2-8e4c-378c0a8f0672']

In [27]:
retriever.get_relevant_documents('영어가 좋아요')

[Document(page_content='한국어는 훌륭합니다', metadata={'last_accessed_at': datetime.datetime(2024, 2, 15, 1, 55, 10, 670228), 'created_at': datetime.datetime(2024, 2, 15, 1, 55, 8, 670931), 'buffer_idx': 1})]

# TODO

To persist the data, you need to use a different type of DocumentStore, such as PyPDFDocumentStore or MongoDBDocumentStore. These document stores store the data on disk or in a database, so it is not lost when you reconnect Colab.

Here is an example of how to use PyPDFDocumentStore:

 ```python
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitters import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import MyScale, MyScaleSettings
from langchain.docstore.pydocumentstore import PyPDFDocumentStore

# Load documents from a PDF file.
loader = PyPDFLoader("my_documents.pdf")
documents = loader.load()

# Create a PyPDFDocumentStore.
docstore = PyPDFDocumentStore(
    documents=documents,
    text_splitter=CharacterTextSplitter(),
    embeddings=MyScale(MyScaleSettings(embedding_function="openai")),
)

# Add a new document.
new_document = Document(page_content="This is a new document.")
docstore.add_documents(new_document)

# Close the document store to save the data to disk.
docstore.close()
```
Use code with caution
Once you have closed the document store, the data will be saved to disk and you can access it again later, even if you reconnect Colab.

In [ ]:
# 이제 위의 리트리버를 이용해서 로딩한 문서를 넣어준다.
# 다른 경우와는 달리, 위에서 설정했던 스플리터를 사용해서 얘가 다 알아서 잘라주고, 부모-자식간의 연결고리도 마련한다.
# 그래서 시간이 현기증날만큼 조온나 오래걸린다. GPU를 써도 오래걸린다. 와 짜증 진짜...

@trace
# def make_context():
#     retriever.add_documents(documents, ids=None)

# make_context()
# len(list(store.yield_keys()))

In [ ]:
# import logging

# logging.basicConfig()
# logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [ ]:
# unique_docs = retriever_from_llm.get_relevant_documents(query=question)
# len(unique_docs)

In [ ]:
# unique_docs

In [ ]:
https://www.youtube.com/watch?v=wQEl0GGxPcM&t=306